# Time Series Anomaly Detecion: Detection of Anomalous Drops with Limited Features and Sparse Examples in Noisy Highly Periodic Data

Paper is [here](https://static.googleusercontent.com/media/research.google.com/ru//pubs/archive/dfd834facc9460163438b94d53b36f51bb5ea952.pdf), additional information is [here](https://nau-datascience.github.io/Time-Series-Anomaly-Detection).  
Anomaly detection in network traffic. Only two attributes - time stamp and bytes per second sampled at 5 minute interval. In total - 14 different data streams. Each data stream gets its own model.  
Three models are considered - DNN, RNN and LSTM. Not clear how many input features they used, I will use 50 because this is what is used in examples on the page with additional info.  
**Models**  
1. DNN model - DNN Regressor - fully-connected model. In total, 10 layers of 200 neurons each, batch size 200, trained for 1200 steps, single linear output layer
2. RNN model - 10 layer RNN with 75 units in hidden size, batch size 200 at 2500 steps.
3. LSTM - 10 layer model with 70 neurons, single lienar output layer.

Authors report that RNN/LSTM did not perform much better compared to DNN. Summary*:
1. DNN regressor: 3.702000e-04 gFLOPs (forward)
2. RNN mode: 5.347575e-03 gFLOPs (forward)
3. LSTM model: 1.863407e-02 gFLOPs (forward)


*FLOPs - Approximate number of multiply-add operations to perform a task.

In [1]:
import contextlib
import os, sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns.model import Model
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, LSTM, Activation, SimpleRNN

In [2]:
inference = []
training = []

### DNN Regressor

In [3]:
class DNNModel(Model):
    def __init__(self):
        super().__init__('DNNModel')

    def create(self):
        model = models.Sequential([Model.Input((50,))], name=self.name)
        for i in range(10):
                model.add(Model.Dense(200, name='dense{}'.format(i+1)))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [4]:
estimate(DNNModel(), inference, training)

,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(50,)",0.000000e+00,0,50,0.000000,0.000200
1,dense1,"(200,)",1.000000e-05,10200,400,0.040800,0.001600
2,dense2,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
3,dense3,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
4,dense4,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
5,dense5,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
6,dense6,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
7,dense7,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
8,dense8,"(200,)",4.000000e-05,40200,400,0.160800,0.001600
9,dense9,"(200,)",4.000000e-05,40200,400,0.160800,0.001600


### RNN model

In [5]:
class RNNModel(Model):
    def __init__(self):
        super().__init__('RNNModel')

    def create(self):
        model = models.Sequential([Model.Input((50,1))], name=self.name)
        for i in range(9):
                model.add(SimpleRNN(75, return_sequences=True, name='rnn0{}'.format(i+1)))
        model.add(SimpleRNN(75, return_sequences=False, name='rnn10'))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [6]:
estimate(RNNModel(), inference, training)

,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(50, 1)",0.000000e+00,0,50,0.000000,0.000200
1,rnn01 (SimpleRNN),"(50, 75)",2.850000e-04,5775,11250,0.023100,0.045000
2,rnn02 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
3,rnn03 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
4,rnn04 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
5,rnn05 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
6,rnn06 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
7,rnn07 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
8,rnn08 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000
9,rnn09 (SimpleRNN),"(50, 75)",5.625000e-04,11325,11250,0.045300,0.045000


### LSTM model

In [7]:
class LSTMModel(Model):
    def __init__(self):
        super().__init__('LSTMModel')

    def create(self):
        model = models.Sequential([Model.Input((50,1))], name=self.name)
        for i in range(9):
                model.add(LSTM(70, return_sequences=True, name='lstm0{}'.format(i+1)))
        model.add(LSTM(70, return_sequences=False, name='lstm10'))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [9]:
estimate(LSTMModel(), inference, training)

W0522 14:07:11.581046 140265770514176 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f91301d4160>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 14:07:11.902974 140265770514176 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f91301d41d0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 14:07:12.202021 140265770514176 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f90ec09f978>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 14:07:12.501918 140265770514176 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f90e45c0748>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better perf

,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(50, 1)",0.000000e+00,0,50,0.000000,0.000200
1,lstm01 (LSTM),"(50, 70)",9.940000e-04,20160,49000,0.080640,0.196000
2,lstm02 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
3,lstm03 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
4,lstm04 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
5,lstm05 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
6,lstm06 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
7,lstm07 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
8,lstm08 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000
9,lstm09 (LSTM),"(50, 70)",1.960000e-03,39480,49000,0.157920,0.196000


### Summary: inference

In [11]:
printable_dataframe(inference)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,DNNModel,"(50,)",372201,1.488804,0.000370,0.016204
1,RNNModel,"(50, 1)",107776,0.431104,0.005348,0.450204
2,LSTMModel,"(50, 1)",375551,1.502204,0.018634,1.960204


### Summary: training

In [12]:
printable_dataframe(training)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,DNNModel,"(50,)",372201,1.488804,0.001111,0.032408
1,RNNModel,"(50, 1)",107776,0.431104,0.016043,0.900408
2,LSTMModel,"(50, 1)",375551,1.502204,0.055902,3.920408
